In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
'''from https://github.com/metalwhale/hand_tracking'''
import cv2
import time
from examples.mediapipe_tracking.src.hand_tracker_3d import HandTracker3D
import numpy as np
import os
import re
import json
import csv
import datetime

In [3]:
PALM_MODEL_PATH = "examples/mediapipe_tracking/models/palm_detection_without_custom_op.tflite"
LANDMARK_MODEL_PATH = "examples/mediapipe_tracking/models/hand_landmark_3d.tflite"
ANCHORS_PATH = "examples/mediapipe_tracking/models/anchors.csv"
OUTPUT_PATH = "gesture_learning/Fall 2020/ipn/keypoints_data"
VIDEO_PATH = "/Users/Radiance/capstone/ipn/videos"
LABEL_TRAINING_PATH = "/Users/Radiance/capstone/ipn/annotations/Annot_TrainList.txt"
LABEL_TESTING_PATH = "/Users/Radiance/capstone/ipn/annotations/Annot_TestList.txt"

INPUT_FPS = 30
OUTPUT_FPS = 30
FRAME_STEP = INPUT_FPS // OUTPUT_FPS

In [4]:
detector = HandTracker3D(
    PALM_MODEL_PATH,
    LANDMARK_MODEL_PATH,
    ANCHORS_PATH,
    box_shift=0.2,
    box_enlarge=1.3
)

In [7]:
def generate_datasets(video_path, label_path, output_path):
    os.makedirs(OUTPUT_PATH, exist_ok=True)
    with open(os.path.join(OUTPUT_PATH, 'description.txt'), 'w+') as f:
        f.writelines([f"time modified: {datetime.datetime.now()}\n"])
    labelList = list(csv.reader(labelFile))
        subjectNum = int(re.findall(r'(\d+)', subjectName)[0])
        if subjectNum in exclude: 
            continue
        for sceneName in os.listdir(os.path.join(LABEL_PATH, subjectName)):
            if not (sceneName.startswith("Scene") or subjectName.startswith("scene")): continue
            os.makedirs(os.path.join(OUTPUT_PATH, subjectName, sceneName), exist_ok=True)
            for groupEntry in os.scandir(os.path.join(LABEL_PATH, subjectName, sceneName)):
                dataset = []
                # print(groupEntry.name,re.fullmatch(r'Group\d+.csv', groupEntry.name))
                if not re.fullmatch(r'Group\d+.csv', groupEntry.name): continue
                groupNum = int(re.findall(r'Group(\d+).csv', groupEntry.name)[0])
                with open(groupEntry) as labelFile:
                    curFrameIdx = 0
                    rgbPath = os.path.join(VIDEO_PATH, subjectName, sceneName, 'Color', f'rgb{groupNum}.avi')
                    rgbCap = cv2.VideoCapture(rgbPath)
                    labelList = list(csv.reader(labelFile))
                    print(f"Processing subject {subjectName}, scene {sceneName}, group {groupNum}")
                    for i, (label, startFrame, endFrame) in enumerate(labelList):
                        try:
                            label = int(label)
                            if label not in labels_to_use: continue
                            videoKeypoints = []
                            for nextFrameIdx in range(int(startFrame), int(endFrame), FRAME_STEP):
                                while curFrameIdx < nextFrameIdx:
                                    hasRGBFrame, rgbFrame = rgbCap.read()
                                    curFrameIdx += 1
                                image = cv2.cvtColor(rgbFrame, cv2.COLOR_BGR2RGB)
                                points = detector.detect_rotated_joints(image)
                                # visualize_keypoints(rgbFrame, points, connections)
                                if points is None:
                                    videoKeypoints.append([])
                                else:
                                    videoKeypoints.append(points.tolist())
                            if len(videoKeypoints):
                                dataset.append({'label': label, 'keypoints': videoKeypoints})
                        except Exception as e:
                            print(e)
                    rgbCap.release()
                if not dataset:
                    continue
                with open(os.path.join(OUTPUT_PATH, subjectName, sceneName, f'group{groupNum}.json'), 'w+') as dataFile:
                    json.dump(dataset, dataFile)
                    print(f"Data written to {os.path.abspath(dataFile.name)}")
generate_datasets([1,2])

 subject47, scene Scene6, group 5
Data written to /Users/Radiance/capstone/assistive-mouse/gesture_learning/Fall 2020/data_30/subject47/Scene6/group5.json
Processing subject subject47, scene Scene6, group 6
Data written to /Users/Radiance/capstone/assistive-mouse/gesture_learning/Fall 2020/data_30/subject47/Scene6/group6.json
Processing subject subject47, scene Scene6, group 7
Processing subject subject48, scene Scene1, group 1
Data written to /Users/Radiance/capstone/assistive-mouse/gesture_learning/Fall 2020/data_30/subject48/Scene1/group1.json
Processing subject subject48, scene Scene1, group 2
Processing subject subject48, scene Scene1, group 3
Data written to /Users/Radiance/capstone/assistive-mouse/gesture_learning/Fall 2020/data_30/subject48/Scene1/group3.json
Processing subject subject48, scene Scene1, group 4
Data written to /Users/Radiance/capstone/assistive-mouse/gesture_learning/Fall 2020/data_30/subject48/Scene1/group4.json
Processing subject subject48, scene Scene1, group